In [110]:
import pandas as pd
import numpy as np
import time
import gc
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, GlobalMaxPooling2D, LSTM, MaxPooling2D, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC

In [2]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            15Gi       1.7Gi        11Gi       260Mi       2.3Gi        13Gi
Swap:          976Mi          0B       976Mi


In [3]:
# This df will be a row-concatenated version of the same channels
# baseline = pd.read_csv('../../baseline.csv', delimiter=',')
# baseline.drop('Unnamed: 0', inplace=True, axis=1)

In [3]:
# del occ0
# del occ1
# del fefF3
# del fefF4
# del fefF7
# del fefF8
# del temT7
# del temT8
# del pfcAF3
# del pfcAF4
# del motFC5
# del motFC6
# del parP7
# del parP8

gc.collect()

217

In [3]:
# loading dataframes for each channel

# occ0 = pd.read_csv('../data/occ0Exp.csv', delimiter=',')
# occ1 = pd.read_csv('../data/occ1Exp.csv', delimiter=',')
# fefF3 = pd.read_csv('../data/fefF3Exp.csv', delimiter=',')
# fefF4 = pd.read_csv('../data/fefF4Exp.csv', delimiter=',')
# fefF7 = pd.read_csv('../data/fefF7Exp.csv', delimiter=',')
# fefF8 = pd.read_csv('../data/fefF8Exp.csv', delimiter=',')
# temT7 = pd.read_csv('../data/temT7Exp.csv', delimiter=',')
# temT8 = pd.read_csv('../data/temT8Exp.csv', delimiter=',')
# pfcAF3 = pd.read_csv('../data/pfcAF3Exp.csv', delimiter=',')
# pfcAF4 = pd.read_csv('../data/pfcAF4Exp.csv', delimiter=',')
# motFC5 = pd.read_csv('../data/motFC5Exp.csv', delimiter=',')
# motFC6 = pd.read_csv('../data/motFC6Exp.csv', delimiter=',')
# parP7 = pd.read_csv('../data/parP7Exp.csv', delimiter=',')
# parP8 = pd.read_csv('../data/parP8Exp.csv', delimiter=',')

# loading same datasets expanded with filter processing

# occ0 = pd.read_csv('../data/occ0Proc.csv', delimiter=',')
occ1 = pd.read_csv('../data/occ1Proc.csv', delimiter=',') 
# fefF3 = pd.read_csv('../data/fefF3Proc.csv', delimiter=',')
fefF4 = pd.read_csv('../data/fefF4Proc.csv', delimiter=',')
# fefF7 = pd.read_csv('../data/fefF7Proc.csv', delimiter=',')
fefF8 = pd.read_csv('../data/fefF8Proc.csv', delimiter=',') 
# temT7 = pd.read_csv('../data/temT7Proc.csv', delimiter=',')
# temT8 = pd.read_csv('../data/temT8Proc.csv', delimiter=',')
# pfcAF3 = pd.read_csv('../data/pfcAF3Proc.csv', delimiter=',')
pfcAF4 = pd.read_csv('../data/pfcAF4Proc.csv', delimiter=',')
# motFC5 = pd.read_csv('../data/motFC5Proc.csv', delimiter=',')
motFC6 = pd.read_csv('../data/motFC6Proc.csv', delimiter=',')
# parP7 = pd.read_csv('../data/parP7Proc.csv', delimiter=',')
parP8 = pd.read_csv('../data/parP8Proc.csv', delimiter=',')


In [5]:
# loading only the dataframes from channels positively correlated with code, to save RAM
corrcols = [occ1, fefF4, fefF8, pfcAF4, motFC6, parP8] #removed motFC5 for being a diff shape

code          1.000000

occ0Data     -0.000385

occ1Data      0.001626

fefF3Data    -0.005483

fefF4Data     0.008288

fefF7Data    -0.004799

fefF8Data     0.006907

temT7Data    -0.002657

temT8Data    -0.001037

pfcAF3Data   -0.007512

pfcAF4Data    0.007591

motFC5Data    0.003757

motFC6Data    0.001372

parP7Data    -0.003000

parP8Data     0.002366


In [6]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            15Gi       8.1Gi       1.1Gi       229Mi       6.4Gi       6.9Gi
Swap:          976Mi          0B       976Mi


In [9]:
# dfs = [occ0, occ1, fefF3, fefF4, fefF7, fefF8, 
#        temT7, temT8, pfcAF3, pfcAF4, motFC5, motFC6, parP7, parP8]
for i in corrcols: #dfs:
#     i.drop('Unnamed: 0.1', inplace=True, axis=1)
    i.drop(['Unnamed: 0', 'id', 'size'], inplace=True, axis=1)
    i['data'].astype(float, copy=False)
    print(i.columns)
#     i = i.groupby('event').mean()
#     print(i.columns)

Index(['event', 'channel', 'code', 'data', 'ICAdata', 'SGFdata'], dtype='object')
Index(['id', 'event', 'device', 'channel', 'code', 'size', 'data', 'ICAdata',
       'SGFdata'],
      dtype='object')
Index(['event', 'channel', 'code', 'data', 'ICAdata', 'SGFdata'], dtype='object')
Index(['event', 'channel', 'code', 'data', 'ICAdata', 'SGFdata'], dtype='object')
Index(['event', 'channel', 'code', 'data', 'ICAdata', 'SGFdata'], dtype='object')
Index(['event', 'channel', 'code', 'data', 'ICAdata', 'SGFdata'], dtype='object')


In [10]:
# non loop version of above's last line
# occ1, fefF4, fefF8, pfcAF4, motFC5, motFC6, parP8 for corrcols

# occ0 = occ0.groupby('event').mean()
occ1 = occ1.groupby('event').mean()
# fefF3 = fefF3.groupby('event').mean()
fefF4 = fefF4.groupby('event').mean()
# fefF7 = fefF7.groupby('event').mean()
fefF8 = fefF8.groupby('event').mean()
# temT7 = temT7.groupby('event').mean()
# temT8 = temT8.groupby('event').mean()
# pfcAF3 = pfcAF3.groupby('event').mean()
pfcAF4 = pfcAF4.groupby('event').mean()
# motFC5 = motFC5.groupby('event').mean() #this one has less rows than the rest, why?
motFC6 = motFC6.groupby('event').mean()
# parP7 = parP7.groupby('event').mean()
parP8 = parP8.groupby('event').mean()


In [11]:
# for i in range(0, len(corrcols)): # len(dfs)):
#     print(len(corrcols[i]))#(len(dfs[i]))
    
# print(len(motFC5), len(motFC6))
motFC6

,code,data,ICAdata,SGFdata
event,,,,
67635,6,4211.530572,0.000043,4201.209935
67636,7,4211.852071,0.000044,4200.805726
67637,9,4220.348557,0.000084,4200.404627
67638,9,4209.702797,0.000034,4200.000418
67639,0,4202.767258,0.000001,4199.593100
...,...,...,...,...
132664,-1,4272.974759,0.000333,4268.930860
132665,-1,4268.960336,0.000314,4269.141727
132666,-1,4327.914663,0.000593,4269.352593


In [12]:
for i in corrcols:
    print(max(i.isna().sum()))

0
0
0
0
0
0


In [27]:
!free -h 

               total        used        free      shared  buff/cache   available
Mem:            15Gi       8.5Gi       2.8Gi       188Mi       4.3Gi       6.6Gi
Swap:          976Mi       528Mi       448Mi


In [58]:
# putting the var names in a loop that grabs whole dfs is harder than it seems
# occ1, fefF4, fefF8, pfcAF4, motFC6, parP8 for corrcols

# occ0.rename(columns={'data': 'occ0data'}, inplace=True)
occ1.rename(columns={'data':'occ1data', 'ICAdata':'occ1ICAdata', 'SGFdata':'occ1SGFdata'}, inplace=True)
# fefF3.rename(columns={'data': 'fefF3data'}, inplace=True)
fefF4.rename(columns={'data':'fefF4data', 'ICAdata':'fefF4ICAdata', 'SGFdata':'fefF4SGFdata'}, inplace=True)
# fefF7.rename(columns={'data': 'fefF7data'}, inplace=True)
fefF8.rename(columns={'data':'fefF8data', 'ICAdata':'fefF8ICAdata', 'SGFdata':'fefF8SGFdata'}, inplace=True)
# temT7.rename(columns={'data': 'temT7data'}, inplace=True)
# temT8.rename(columns={'data': 'temT8data'}, inplace=True)
# pfcAF3.rename(columns={'data': 'pfcAF3data'}, inplace=True)
pfcAF4.rename(columns={'data':'pfcAF4data', 'ICAdata':'pfcAF4ICAdata', 'SGFdata':'pfcAF4SGFdata'}, inplace=True)
# motFC5.rename(columns={'data': 'motFC5data'}, inplace=True)
motFC6.rename(columns={'data':'motFC6data', 'ICAdata':'motFC6ICAdata', 'SGFdata':'motFC6SGFdata'}, inplace=True)
# parP7.rename(columns={'data': 'parP7data'}, inplace=True)
parP8.rename(columns={'data':'parP8data', 'ICAdata':'parP8ICAdata', 'SGFdata':'parP8SGFdata'}, inplace=True)


In [14]:
# occ0.columns, occ1.columns, fefF7.columns, fefF8.columns
# # dfs[0] #this wasn't changed with the above functions, it's only used for calling the original datasets as they were read in
# dfsProcessed = [occ0, occ1, fefF3, fefF4, fefF7, fefF8, 
#        temT7, temT8, pfcAF3, pfcAF4, motFC5, motFC6, parP7, parP8]

corrcolsProcessed = [occ1, fefF4, fefF8, pfcAF4, motFC6, parP8] #new vars because the original list has the unprocessed dataframes in memory it seems

In [15]:
full = pd.concat(corrcolsProcessed, axis=1, join='inner')

In [17]:
!free -h # why does the RAM go down a bunch after the above two cells? (at least in corrcols round 12/20)

               total        used        free      shared  buff/cache   available
Mem:            15Gi       6.6Gi       2.9Gi       216Mi       6.0Gi       8.4Gi
Swap:          976Mi       0.0Ki       976Mi


In [18]:
full.columns.duplicated()
# since the concat created duplicate code columns, using this trick to delete them:
#https://stackoverflow.com/questions/14984119/python-pandas-remove-duplicate-columns
full = full.loc[:,~full.columns.duplicated()].copy()

In [19]:
full.dtypes

code               int64
occ1data         float64
occ1ICAdata      float64
occ1SGFdata      float64
id                 int64
size               int64
fefF4data        float64
fefF4ICAdata     float64
fefF4SGFdata     float64
fefF8data        float64
fefF8ICAdata     float64
fefF8SGFdata     float64
pfcAF4data       float64
pcaAF4ICAdata    float64
pfcAF4SGFdata    float64
motFC6data       float64
motFC6ICAdata    float64
motFC6SGFdata    float64
parP8data        float64
parP8ICAdata     float64
parP8SGFdata     float64
dtype: object

In [20]:
full = full.drop(['size', 'id' ], axis=1) # try to make this work in the earlier cell, missed something

In [21]:
full

,code,occ1data,occ1ICAdata,occ1SGFdata,fefF4data,fefF4ICAdata,fefF4SGFdata,fefF8data,fefF8ICAdata,fefF8SGFdata,pfcAF4data,pcaAF4ICAdata,pfcAF4SGFdata,motFC6data,motFC6ICAdata,motFC6SGFdata,parP8data,parP8ICAdata,parP8SGFdata
event,,,,,,,,,,,,,,,,,,,
67635,6,4214.524654,-0.000023,4224.184902,4635.313609,-0.000005,4648.100047,3988.708087,-0.000018,4002.607056,4058.491124,0.000056,4064.567262,4211.530572,0.000043,4201.209935,4227.084812,0.000030,4235.984111
67636,7,4221.477317,-0.000049,4223.634952,4638.408284,0.000006,4647.593850,4005.897436,-0.000065,4001.908551,4066.686390,0.000092,4063.927970,4211.852071,0.000044,4200.805726,4234.327416,-0.000020,4235.375812
67637,9,4221.193910,-0.000048,4223.089233,4652.423878,0.000056,4647.091548,3994.072516,-0.000033,4001.215419,4068.056891,0.000098,4063.293597,4220.348557,0.000084,4200.404627,4237.431891,-0.000041,4234.772192
67638,9,4220.273892,-0.000044,4222.539284,4653.931623,0.000061,4646.585351,3991.991065,-0.000027,4000.516914,4068.580031,0.000100,4062.654305,4209.702797,0.000034,4200.000418,4232.148407,-0.000005,4234.163892
67639,0,4219.587771,-0.000042,4221.985104,4648.289940,0.000041,4646.075261,3991.954635,-0.000027,3999.813036,4062.779092,0.000075,4062.010096,4202.767258,0.000001,4199.593100,4228.664694,0.000019,4233.550914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132664,-1,4214.014423,-0.000021,4220.554161,4551.911057,-0.000302,4585.537579,3997.776442,-0.000043,3972.581650,3905.224359,-0.000614,3939.122349,4272.974759,0.000333,4268.930860,4257.497996,-0.000181,4252.699773
132665,-1,4210.893429,-0.000009,4221.533056,4584.789663,-0.000185,4585.773027,3957.584135,0.000065,3972.782644,3927.640225,-0.000516,3940.105698,4268.960336,0.000314,4269.141727,4250.114182,-0.000129,4253.046906
132666,-1,4252.604166,-0.000164,4222.511951,4642.453926,0.000020,4586.008474,4032.510016,-0.000136,3972.983637,3988.868189,-0.000248,3941.089047,4327.914663,0.000593,4269.352593,4294.162660,-0.000435,4253.394039


In [22]:
corrMatFull = full.corr()
corrMatFull['code']
# check = pd.read_csv('../data/occ1Exp.csv', delimiter=',')
# print(check.corr()) # code correlates to data in this channel by 0.001562 
# del check 

# check = pd.read_csv('../data/fefF7Exp.csv', delimiter=',')
# print(check.corr()) # code correlates to data in this channel by -0.004390.
# del check 

# check = pd.read_csv('../data/occ0Exp.csv', delimiter=',')
# print(check.corr()) # code correlates to data in this channel by -0.000308 
# del check 

# check = pd.read_csv('../data/fefF8Exp.csv', delimiter=',')
# print(check.corr()) # code correlates to data in this channel by 0.006184
# del check 

code             1.000000
occ1data         0.001626
occ1ICAdata     -0.001626
occ1SGFdata      0.000756
fefF4data        0.008288
fefF4ICAdata     0.008288
fefF4SGFdata     0.009824
fefF8data        0.006907
fefF8ICAdata    -0.006907
fefF8SGFdata     0.003993
pfcAF4data       0.007591
pcaAF4ICAdata    0.007591
pfcAF4SGFdata    0.020014
motFC6data       0.001372
motFC6ICAdata    0.001372
motFC6SGFdata   -0.010552
parP8data        0.002366
parP8ICAdata    -0.002366
parP8SGFdata    -0.008179
Name: code, dtype: float64

In [20]:
# full.plot(y=['occ0Data', 'occ1Data', 'fefF7Data', 'fefF8Data'], alpha=.7 ); 

In [55]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            15Gi       8.6Gi       2.7Gi       172Mi       4.3Gi       6.5Gi
Swap:          976Mi       524Mi       452Mi


--------------
## Baseline Model: Only O1, O2, F7, F8; No Preprocessing
## Baseline+ : Basline+Preprocessing

--------------

&nbsp;


In [60]:
full.columns

Index(['code', 'occ1data', 'occ1ICAdata', 'occ1SGFdata', 'fefF4data',
       'fefF4ICAdata', 'fefF4SGFdata', 'fefF8data', 'fefF8ICAdata',
       'fefF8SGFdata', 'pfcAF4data', 'pcaAF4ICAdata', 'pfcAF4SGFdata',
       'motFC6data', 'motFC6ICAdata', 'motFC6SGFdata', 'parP8data',
       'parP8ICAdata', 'parP8SGFdata'],
      dtype='object')

In [183]:
X = full.drop('code', axis=1)
XExperiment = full.drop(['code', 'occ1data', 'fefF4data',
                         'fefF8data', 'pfcAF4data', 'motFC6data', 'parP8data',
                         'occ1SGFdata', 'fefF4SGFdata','fefF8SGFdata', 'pfcAF4SGFdata', #TODO change to pfc
                         'motFC6SGFdata', 'parP8SGFdata'], axis=1) 
y = full['code']
# print(X.columns, X.iloc[0], y.iloc[0], X.shape, y.shape)

In [184]:
XTrain, XTest, yTrain, yTest = train_test_split(XExperiment, y)

In [136]:
!cowsay brains \& NN algorithms are complex!

 _____________________________________
< brains & NN algorithms are complex! >
 -------------------------------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||


In [145]:
XTrain.shape, XTest.shape, yTrain.shape, yTest.shape

(TensorShape([48775, 1, 6, 1]), (16259, 6), (48775,), (16259,))

In [138]:
# for col in X.columns:
#     print(len(X[col]))

# the error is happening only with the Savitsky-Golay filtered data

X['fefF4SGFdata'] - X['fefF4data'] 

event
67635     12.786438
67636      9.185567
67637     -5.332330
67638     -7.346272
67639     -2.214679
            ...    
132664    33.626522
132665     0.983364
132666   -56.445452
132667   -11.542537
132668   -19.930888
Length: 65034, dtype: float64

In [139]:
XTrain = tf.reshape(XTrain, (48775, 1, XTrain.shape[1], 1))
ohe = OneHotEncoder(sparse=False)
yTrainOHE = ohe.fit_transform(yTrain.to_numpy().reshape(-1,1))

# ohe0 = OneHotEncoder(sparse=False)
# yTestOHE = ohe0.fit(yTrain.to_numpy().reshape(-1,1)) # Daniel said to do this but I can't remember why

In [140]:
# del model
# gc.collect()

In [141]:
model = Sequential()

In [142]:
model.add(Conv2D(filters=64, kernel_size=1, activation='relu', input_shape=(1,6,1)))
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Conv2D(filters=64, kernel_size=1, activation='relu'))
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Conv2D(filters=64, kernel_size=1, activation='relu'))
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(11, activation='softmax'))

# model.add(LSTM(7, input_shape=(1,14,1), activation='tanh', 
#                recurrent_activation='sigmoid'))

In [143]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 1, 6, 64)          128       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 1, 6, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 1, 6, 64)          4160      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 1, 6, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 1, 6, 64)          4160      
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 1, 6, 64)        

In [149]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
results = model.fit(XTrain, yTrainOHE, batch_size=20,epochs=30, verbose=1)

Epoch 1/30
2439/2439 [==============================] - 7s 2ms/step - loss: 2.3146 - accuracy: 0.1004
Epoch 2/30
2439/2439 [==============================] - 6s 3ms/step - loss: 2.3143 - accuracy: 0.1001
Epoch 3/30
2439/2439 [==============================] - 6s 2ms/step - loss: 2.3143 - accuracy: 0.1008
Epoch 4/30
2439/2439 [==============================] - 6s 2ms/step - loss: 2.3143 - accuracy: 0.1002
Epoch 5/30
2439/2439 [==============================] - 6s 2ms/step - loss: 2.3143 - accuracy: 0.0989
Epoch 6/30
2439/2439 [==============================] - 6s 2ms/step - loss: 2.3143 - accuracy: 0.1016
Epoch 7/30
2439/2439 [==============================] - 6s 2ms/step - loss: 2.3143 - accuracy: 0.1021
Epoch 8/30
2439/2439 [==============================] - 6s 2ms/step - loss: 2.3143 - accuracy: 0.0996
Epoch 9/30
2439/2439 [==============================] - 6s 2ms/step - loss: 2.3143 - accuracy: 0.1007
Epoch 10/30
2439/2439 [==============================] - 6s 2ms/step - loss: 2.314

Best accuracy for the unprocessed signal with all channels was about .1020

In 6/14 channels, adding SGF data was about the same

In 6/14 channels, adding SGF and dropping raw data resulted in really low accuracy and NaN loss

In 6/14 channels, ICA only had a accuracy of about .1000

6/14 channels, unproccessed+SGF+ICA, 22,059 params: NaN loss again, what's wrong with the data? woe be the futility of mine ways

6/14 channels, SGF only, 9,771 params, accuracy still around .1000

6/14 channels, raw, 9,771 params, accuracy about .1000, so the .0020 increase was only from the extra channels. Processing doesn't seem to change the accuracy at all.

-[] TODO: OHE test, validation set, make more complicated models

### Support Vector Machine Attempt

In [109]:
svc = SVC()
svc.fit(XTrain, yTrain)

SVC()

In [116]:
svc.score(XTest, yTest)

0.10234331754720463

A quick support vector classifier ended up getting the same accuracy, 0.10234331754720463

In [197]:
XTrain.to_numpy().reshape(-1, 1).flatten(), yTrain.to_numpy().reshape(-1, 1)

(array([ 1.51991153e-05, -1.64019496e-04,  6.11943053e-05, ...,
         4.13683698e-05,  1.26545442e-04,  1.39856091e-05]),
 array([[2],
        [3],
        [0],
        ...,
        [8],
        [2],
        [0]]))

### Echo State Network Attempt

In [190]:
import reservoirpy as rpy
from reservoirpy.nodes import Reservoir, Ridge, FORCE, ESN

readout = Ridge(ridge=1e-7)
train_states = reservoir.run(XTrain.to_numpy().reshape(-1, 1), reset=True)
# train_statesy = reservoir.run(yTrain.to_numpy().reshape(-1, 1), reset=True)
readout = readout.fit(train_states, yTrain.to_numpy().reshape(-1, 1), warmup=10)
test_states = reservoir.run(XTest.to_numpy().reshape(-1, 1))
yPred = readout.run(test_states)

reservoir = Reservoir(100, lr=0.5, sr=0.9)
ridge = Ridge(ridge=1e-7)

esn_model = reservoir >> ridge

esn_model = esn_model.fit(XTrain.to_numpy().reshape(-1, 1), yTrain.to_numpy().reshape(-1, 1).to_numpy().reshape(-1, 1), warmup=10)
print(reservoir.is_initialized, readout.is_initialized, readout.fitted)

yPred = esn_model.run(XTest.to_numpy().reshape(-1, 1))


ValueError: cannot reshape array of size 292650 into shape (48775,1)

While it was nice to learn how to set one of these up, I realized after a lot of tuning the reservoirpy library doesn't seem to have good interop with pandas. Might have interesting results with 1D timestamp and 1D value arrays, but not a multidimensional dataframe

In [209]:
readout1 = Ridge(ridge=1e-7)
train_states1 = reservoir.run(full['pfcAF4SGFdata'].to_numpy().reshape(-1, 1), reset=True)
# train_statesy = reservoir.run(yTrain.to_numpy().reshape(-1, 1), reset=True)
readout1 = readout1.fit(train_states1, full['code'].to_numpy().reshape(-1, 1), warmup=10)
test_states1 = reservoir.run((full['pfcAF4SGFdata'].to_numpy().reshape(-1, 1)))
# yPred1 = readout.run(test_states1)

reservoir1 = Reservoir(100, lr=0.5, sr=0.9)
ridge1 = Ridge(ridge=1e-7)

esn_model1 = reservoir >> ridge

esn_model1 = esn_model1.fit(full['pfcAF4SGFdata'].to_numpy().reshape(-1, 1), 
                             full['code'].to_numpy().reshape(-1, 1) ,warmup=10)
print(reservoir.is_initialized, readout.is_initialized, readout.fitted)

yPred1 = esn_model.run((full['pfcAF4SGFdata'].to_numpy().reshape(-1, 1)))

Running Model-7:   0%|          | 0/1 [00:00<?, ?it/s]
Running Model-7: 608it [00:00, 6077.69it/s]           
Running Model-7: 1184it [00:00, 5977.77it/s]
Running Model-7: 1774it [00:00, 5953.20it/s]
Running Model-7: 2340it [00:00, 5861.04it/s]
Running Model-7: 2958it [00:00, 5950.73it/s]
Running Model-7: 3509it [00:00, 5809.06it/s]
Running Model-7: 4114it [00:00, 5877.67it/s]
Running Model-7: 4698it [00:00, 5864.64it/s]
Running Model-7: 5266it [00:00, 5805.29it/s]
Running Model-7: 5856it [00:01, 5832.72it/s]
Running Model-7: 6438it [00:01, 5827.26it/s]
Running Model-7: 7041it [00:01, 5886.27it/s]
Running Model-7: 7640it [00:01, 5915.91it/s]
Running Model-7: 8228it [00:01, 5904.99it/s]
Running Model-7: 8820it [00:01, 5909.08it/s]
Running Model-7: 9416it [00:01, 5922.98it/s]
Running Model-7: 10023it [00:01, 5964.50it/s]
Running Model-7: 10635it [00:01, 6009.18it/s]
Running Model-7: 11236it [00:01, 5912.81it/s]
Running Model-7: 11828it [00:02, 5818.73it/s]
Running Model-7: 12411it [00:02

Fitting node Ridge-19...


Running Model-6: 1195it [00:00, 5902.90it/s]          

True False False


Running Model-6: 65034it [00:10, 6031.98it/s]


In [220]:
set(yPred1.flatten()), (mean())

{4.494217014863551}